In [1]:
! pip install --upgrade deepforest albumentations PyYAML
from deepforest import main
from deepforest import get_data
import matplotlib.pyplot as plt
from IPython.display import clear_output
from pathlib import Path
import pandas as pd
import json
clear_output()


In [2]:
m = main.deepforest()
m.use_release()

Reading config file: /home/junior/anaconda3/envs/myenv/lib/python3.12/site-packages/deepforest/data/deepforest_config.yml


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


No validation file provided. Turning off validation loop
Model from DeepForest release https://github.com/weecology/DeepForest/releases/tag/1.0.0 was already downloaded. Loading model from file.
Loading pre-built model: https://github.com/weecology/DeepForest/releases/tag/1.0.0


In [3]:
def create_coco_annotation_from_df(df : pd.core.frame.DataFrame, img_size : int, output_path: str):
    coco = {
        "images": [],
        "annotations": [],
        "categories": []
    }
    
    image_id = 0
    annotation_id = 0
    category_id_map = {}
    
    unique_images = df['image_path'].unique()
    
    for image_path in unique_images:
        image_data = df[df['image_path'] == image_path]
        coco["images"].append({
            "id": image_id,
            "file_name": image_path,
            "width": img_size,
            "height": img_size
        })
        
        for _, row in image_data.iterrows():
            # use row['score'] comme filtre
            label = row["label"] # not mandatory because deepforest model detect only one class
            if label not in category_id_map:
                category_id = len(category_id_map)
                category_id_map[label] = category_id
                coco["categories"].append({
                    "id": category_id,
                    "name": label
                })
            
            coco["annotations"].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": category_id_map[label],
                "bbox": [row["xmin"], row["ymin"], row["xmax"] - row["xmin"], row["ymax"] - row["ymin"]],
                "area": (row["xmax"] - row["xmin"]) * (row["ymax"] - row["ymin"]),
                "iscrowd": 0
            })
            annotation_id += 1
        
        image_id += 1
    
    with open(output_path, "w") as outfile:
        json.dump(coco, outfile, indent=4)

In [13]:
IMAGES_PATH = "hand_labeled/"
LABELS_PATH = "im_labels_2/"
! mkdir {LABELS_PATH}
output_dir = Path(LABELS_PATH)

def annotate(img_size : int, overlap : float = 0.25):
  for i, file_path in enumerate(Path(IMAGES_PATH).glob("*")):
    if file_path.suffix.lower() in [".jpg", ".jpeg"]:
      raster = m.predict_tile(IMAGES_PATH+file_path.name, return_plot = True, patch_size=min(img_size, 400),patch_overlap=overlap)
      output_path = output_dir / f"{file_path.name.split('.')[0]}.jpeg"
      plt.imsave(output_path, raster)
      #create_coco_annotation_from_df(raster, img_size, output_path)
      '''
      plt.figure(figsize=(15, 15))
      plt.subplot(5, 5, i+1)
      plt.xticks([])
      plt.yticks([])
      plt.grid(False)
      if img is not None:
        plt.imshow(img[:,:,::-1])
      '''
  clear_output()
  print("Annotations done")   
  #plt.show()

In [14]:
annotate(1024, 0.25)

Annotations done


In [ ]:
a = m.predict_tile("images/"+"2_29.jpeg", return_plot = True, patch_size=256,patch_overlap=0.25)
plt.imshow(a)